# 분석4: 뉴스 감성 영향 분석

## 목적
뉴스 긍정/부정이 집값에 미친 영향 분석

## 분석 대상
- **지역**: 강남구, 서초구, 송파구 (강남3구)
- **기간**: 2023년 ~ 2025년 (3년)
- **뉴스 출처**: 중앙일보 부동산 뉴스
- **가격 출처**: 서울 열린데이터 광장 API

## 분석 내용
- 감성지수 vs 실거래가 상관관계
- 긍정/부정 뉴스 증가 시점 전후 비교
- 시차 분석 (Lag Analysis)
- 구별 민감도 차이

## 핵심 질문
1. 긍정 뉴스가 많으면 집값이 오르는가?
2. 반응 속도는 얼마나 되는가?
3. 어느 구가 뉴스에 가장 민감한가?

## 1. 라이브러리 설치 및 임포트

In [ ]:
# 필요한 라이브러리 설치 (처음 실행 시 주석 해제)
# !pip install pandas numpy matplotlib seaborn plotly scipy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from scipy.stats import pearsonr, spearmanr, linregress
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False

## 2. 데이터 로드

### 2-1. 뉴스 크롤링 데이터 로드 (중앙일보)

In [ ]:
# ============================================
# 실제 데이터 로드 (팀원 크롤링 데이터)
# ============================================
# news_df = pd.read_csv('../data/news_data.csv')

# ============================================
# 샘플 데이터 생성 (테스트용 - 실제 데이터로 교체 필요)
# ============================================
np.random.seed(42)

# 3년치 데이터: 2023-01-01 ~ 2025-12-31
dates = pd.date_range(start='2023-01-01', end='2025-12-31', freq='D')

# 강남3구 + 중앙일보
REGIONS = ['강남구', '서초구', '송파구']
SOURCE = '중앙일보'

# 샘플 뉴스 데이터 생성 (감성 키워드 포함)
sample_titles = [
    '강남 아파트 가격 급등, 신고가 경신',
    '서초구 재건축 호재로 매수세 증가',
    '송파구 집값 하락세, 거래절벽 지속',
    '금리인상 여파로 부동산 시장 침체',
    '강남3구 청약 열기 뜨거워',
    'GTX 개통 기대감에 집값 상승',
    '종부세 부담으로 매물 증가',
    '부동산 규제 완화 기대감 확산',
    '강남 재건축 분양가 신고가 기록',
    '전세 시장 불안, 역전세 우려',
    '서초구 학군 인기로 매수 문의 증가',
    '송파구 신축 아파트 완판 행진',
    '부동산 경기침체 장기화 전망',
    '강남 프리미엄 아파트 투자 수익 증가',
    '대출 규제로 매수세 위축'
]

n_news = 1500
news_df = pd.DataFrame({
    'date': np.random.choice(dates, n_news),
    'title': np.random.choice(sample_titles, n_news),
    'content': np.random.choice(sample_titles, n_news),
    'source': [SOURCE] * n_news,
    'region': np.random.choice(REGIONS, n_news)
})

news_df['date'] = pd.to_datetime(news_df['date'])
news_df = news_df.sort_values('date').reset_index(drop=True)

print(f"뉴스 데이터: {len(news_df):,}건")
news_df.head()

### 2-2. 뉴스 데이터 기본 정보 확인

In [ ]:
# ============================================
# 뉴스 데이터 기본 정보 확인
# ============================================

print("="*60)
print("📰 뉴스 데이터 기본 정보")
print("="*60)

print(f"\n전체 뉴스: {len(news_df):,}건")
print(f"기간: {news_df['date'].min().strftime('%Y-%m-%d')} ~ {news_df['date'].max().strftime('%Y-%m-%d')}")

# 월별 평균 뉴스 건수
months_count = news_df['date'].dt.to_period('M').nunique()
print(f"분석 기간: {months_count}개월")
print(f"월평균 뉴스: {len(news_df)/months_count:.0f}건")

# 구별 뉴스 건수
print(f"\n[구별 뉴스 건수]")
print(news_df['region'].value_counts())

# 결측치 확인
print(f"\n[결측치 확인]")
print(news_df.isna().sum())

### 2-3. 실거래가 데이터 로드

In [ ]:
# ============================================
# 실제 데이터 로드 (팀원 API 데이터)
# ============================================
# price_df = pd.read_csv('../data/estate_price.csv')

# ============================================
# 샘플 실거래가 데이터 생성 (테스트용)
# ============================================

# 3년치 월별 데이터
months = pd.date_range(start='2023-01-01', end='2025-12-31', freq='M')

# 강남3구 평균 시세 (억원 기준)
base_prices = {
    '강남구': 25,
    '서초구': 22,
    '송파구': 18
}

price_data = []
for region in REGIONS:
    base = base_prices[region]
    for i, month in enumerate(months):
        price = base + (i * 0.08) + np.random.normal(0, 0.3)
        price_data.append({
            'date': month,
            'region': region,
            'avg_price': round(price, 2)
        })

price_df = pd.DataFrame(price_data)
price_df['date'] = pd.to_datetime(price_df['date'])

print(f"실거래가 데이터: {len(price_df):,}건")
print(f"기간: {price_df['date'].min().strftime('%Y-%m')} ~ {price_df['date'].max().strftime('%Y-%m')}")
price_df.head()

## 3. 감성 분석 (Sentiment Analysis)

### 3-1. 키워드 기반 감성 분석

In [ ]:
# 부동산 관련 감성 키워드 사전
positive_keywords = [
    '상승', '급등', '호재', '활황', '회복', '상승세', '강세', '인기', '호황',
    '투자', '수익', '개발', '재건축', '호가', '신고가', '프리미엄', '완판',
    '청약', '당첨', '분양', '매수', '입주', '교통호재', 'GTX', '개통',
    '학군', '인프라', '공원', '편의시설', '안정', '전망좋음'
]

negative_keywords = [
    '하락', '급락', '폭락', '침체', '불황', '하락세', '약세', '위기',
    '거래절벽', '미분양', '매물증가', '매도', '손실', '역전세', '깡통전세',
    '규제', '대출', '금리', '세금', '종부세', '양도세', '취득세',
    '투기', '버블', '과열', '조정', '경기침체', '불안', '우려'
]

def calculate_sentiment_score(text):
    """키워드 기반 감성 점수 계산 (-1 ~ 1)"""
    if pd.isna(text):
        return 0
    
    text = str(text)
    pos_count = sum(1 for word in positive_keywords if word in text)
    neg_count = sum(1 for word in negative_keywords if word in text)
    
    total = pos_count + neg_count
    if total == 0:
        return 0
    
    return (pos_count - neg_count) / total

# 감성 점수 계산 (제목 + 본문)
news_df['sentiment_score'] = (
    news_df['title'].apply(calculate_sentiment_score) * 0.6 +
    news_df['content'].apply(calculate_sentiment_score) * 0.4
)

# 정규화
news_df['sentiment_score'] = news_df['sentiment_score'].clip(-1, 1)

# 감성 레이블 추가
news_df['sentiment_label'] = news_df['sentiment_score'].apply(
    lambda x: '긍정' if x > 0.1 else ('부정' if x < -0.1 else '중립')
)

print("📊 감성 분포:")
print(news_df['sentiment_label'].value_counts())
print(f"\n평균 감성 점수: {news_df['sentiment_score'].mean():.4f}")

### 3-2. 감성 키워드 빈도 분석

In [ ]:
# ============================================
# 감성 키워드 빈도 분석
# ============================================

# 긍정 키워드 빈도
pos_counts = {}
for keyword in positive_keywords:
    count = news_df['title'].str.contains(keyword, na=False).sum()
    count += news_df['content'].str.contains(keyword, na=False).sum()
    if count > 0:
        pos_counts[keyword] = count

# 부정 키워드 빈도
neg_counts = {}
for keyword in negative_keywords:
    count = news_df['title'].str.contains(keyword, na=False).sum()
    count += news_df['content'].str.contains(keyword, na=False).sum()
    if count > 0:
        neg_counts[keyword] = count

# 데이터프레임 변환
pos_df = pd.DataFrame(pos_counts.items(), columns=['keyword', 'count'])
pos_df = pos_df.sort_values('count', ascending=False).head(10)

neg_df = pd.DataFrame(neg_counts.items(), columns=['keyword', 'count'])
neg_df = neg_df.sort_values('count', ascending=False).head(10)

# 긍정 vs 부정 총합 비교
total_pos = sum(pos_counts.values()) if pos_counts else 0
total_neg = sum(neg_counts.values()) if neg_counts else 0

print(f"긍정 키워드 총 등장: {total_pos:,}회")
print(f"부정 키워드 총 등장: {total_neg:,}회")
print(f"긍정/부정 비율: {total_pos/(total_neg+1):.2f}")

In [ ]:
# 시각화 - 긍정 키워드 TOP 10
if len(pos_df) > 0:
    fig_pos = px.bar(
        pos_df,
        x='keyword',
        y='count',
        title='📈 긍정 키워드 TOP 10',
        color_discrete_sequence=['#4CAF50'],
        text='count'
    )
    fig_pos.update_traces(textposition='outside')
    fig_pos.update_layout(template='plotly_white', height=400)
    fig_pos.show()

# 시각화 - 부정 키워드 TOP 10
if len(neg_df) > 0:
    fig_neg = px.bar(
        neg_df,
        x='keyword',
        y='count',
        title='📉 부정 키워드 TOP 10',
        color_discrete_sequence=['#F44336'],
        text='count'
    )
    fig_neg.update_traces(textposition='outside')
    fig_neg.update_layout(template='plotly_white', height=400)
    fig_neg.show()

## 4. 월별 감성지수 집계

In [ ]:
# 월별 집계
news_df['year_month'] = news_df['date'].dt.to_period('M')

# 전체 월별 감성지수
monthly_sentiment = news_df.groupby('year_month').agg({
    'sentiment_score': ['mean', 'std', 'count'],
    'sentiment_label': lambda x: (x == '긍정').sum() / len(x) * 100
}).reset_index()

monthly_sentiment.columns = ['year_month', 'avg_sentiment', 'sentiment_std', 'news_count', 'positive_ratio']
monthly_sentiment['date'] = monthly_sentiment['year_month'].dt.to_timestamp()

# 구별 월별 감성지수
regional_sentiment = news_df.groupby(['year_month', 'region']).agg({
    'sentiment_score': 'mean',
    'sentiment_label': lambda x: (x == '긍정').sum() / len(x) * 100
}).reset_index()

regional_sentiment.columns = ['year_month', 'region', 'avg_sentiment', 'positive_ratio']
regional_sentiment['date'] = regional_sentiment['year_month'].dt.to_timestamp()

print("📅 월별 감성지수:")
monthly_sentiment.head(10)

## 5. 감성지수 vs 실거래가 상관관계 분석

In [ ]:
# 데이터 병합
price_df['year_month'] = price_df['date'].dt.to_period('M')
price_df['date'] = price_df['year_month'].dt.to_timestamp()

# 전체 평균 가격 (강남3구 평균)
monthly_price = price_df.groupby('year_month')['avg_price'].mean().reset_index()
monthly_price['date'] = monthly_price['year_month'].dt.to_timestamp()

# 감성지수와 가격 병합
merged_df = pd.merge(
    monthly_sentiment[['date', 'avg_sentiment', 'positive_ratio', 'news_count']],
    monthly_price[['date', 'avg_price']],
    on='date',
    how='inner'
)

# 상관계수 계산
if not merged_df.empty and len(merged_df) > 5:
    corr_sentiment_price, p_value = pearsonr(
        merged_df['avg_sentiment'].dropna(), 
        merged_df['avg_price'].dropna()
    )
else:
    corr_sentiment_price, p_value = 0, 1.0

print("="*50)
print("📊 감성지수 vs 집값 상관관계 분석")
print("="*50)
print(f"상관계수 (Pearson r): {corr_sentiment_price:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"통계적 유의성: {'✅ 유의함 (p < 0.05)' if p_value < 0.05 else '❌ 유의하지 않음'}")

## 6. 시차 분석 (Lag Analysis)

뉴스 감성이 집값에 영향을 미치기까지 시간이 걸리는지 분석

In [ ]:
def calculate_lag_correlation(df, sentiment_col, price_col, max_lag=6):
    """시차별 상관계수 계산"""
    results = []
    
    for lag in range(-max_lag, max_lag + 1):
        if lag < 0:
            sentiment = df[sentiment_col].iloc[-lag:].values
            price = df[price_col].iloc[:lag].values
        elif lag > 0:
            sentiment = df[sentiment_col].iloc[:-lag].values
            price = df[price_col].iloc[lag:].values
        else:
            sentiment = df[sentiment_col].values
            price = df[price_col].values
        
        if len(sentiment) > 5 and len(price) > 5:
            corr, p_val = pearsonr(sentiment, price)
            results.append({
                'lag': lag,
                'correlation': corr,
                'p_value': p_val,
                'significant': '✅' if p_val < 0.05 else ''
            })
    
    return pd.DataFrame(results)

# 시차 분석 실행
lag_results = calculate_lag_correlation(merged_df, 'avg_sentiment', 'avg_price', max_lag=6)

print("📊 시차별 상관계수:")
print("(양수 lag = 감성이 가격에 선행, 음수 lag = 가격이 감성에 선행)")
print(lag_results.to_string(index=False))

# 최적 시차 찾기
if not lag_results.empty and lag_results['correlation'].notna().any():
    best_lag = lag_results.loc[lag_results['correlation'].abs().idxmax()]
    print(f"\n🎯 가장 강한 상관관계: {int(best_lag['lag'])}개월 시차 (r={best_lag['correlation']:.4f})")
else:
    best_lag = pd.Series({'lag': 0, 'correlation': 0})
    print("\n유의미한 상관관계를 찾을 수 없습니다.")

## 7. 긍정/부정 뉴스 증가 시점 전후 비교

In [ ]:
# ============================================
# 감성 급변 시점 분석 (전후 비교)
# ============================================

# 전월 대비 감성 변화량 계산
monthly_sentiment_sorted = monthly_sentiment.sort_values('date').copy()
monthly_sentiment_sorted['sentiment_change'] = monthly_sentiment_sorted['avg_sentiment'].diff()

# 감성 급등 시점 (상위 3개월)
sentiment_spikes = monthly_sentiment_sorted.nlargest(3, 'sentiment_change')

# 감성 급락 시점 (하위 3개월)
sentiment_drops = monthly_sentiment_sorted.nsmallest(3, 'sentiment_change')

print("="*60)
print("📊 감성 급변 시점 분석")
print("="*60)

print("\n🔺 감성 급등 TOP 3:")
for idx, row in sentiment_spikes.iterrows():
    print(f"   {row['date'].strftime('%Y-%m')}: 변화량 {row['sentiment_change']:+.3f}")

print("\n🔻 감성 급락 TOP 3:")
for idx, row in sentiment_drops.iterrows():
    if pd.notna(row['sentiment_change']):
        print(f"   {row['date'].strftime('%Y-%m')}: 변화량 {row['sentiment_change']:+.3f}")

In [ ]:
# 감성 급등 시점의 집값 변화 분석
print("\n" + "="*60)
print("📈 감성 급등 시점 전후 집값 변화")
print("="*60)

for idx, row in sentiment_spikes.iterrows():
    event_date = row['date']
    print(f"\n▶ {event_date.strftime('%Y-%m')} (감성 변화: {row['sentiment_change']:+.3f})")
    
    # 해당 시점 전후 데이터
    event_data = merged_df[
        (merged_df['date'] >= event_date - pd.DateOffset(months=2)) &
        (merged_df['date'] <= event_date + pd.DateOffset(months=2))
    ].copy()
    
    if len(event_data) > 0:
        before = event_data[event_data['date'] < event_date]['avg_price'].mean()
        after = event_data[event_data['date'] > event_date]['avg_price'].mean()
        current = event_data[event_data['date'] == event_date]['avg_price'].values
        
        if len(current) > 0 and pd.notna(before) and pd.notna(after):
            current = current[0]
            print(f"   이전 2개월 평균: {before:.2f}억")
            print(f"   당월: {current:.2f}억")
            print(f"   이후 2개월 평균: {after:.2f}억")
            print(f"   변화: {after - before:+.2f}억 ({(after-before)/before*100:+.1f}%)")

In [ ]:
# 시각화 - 감성 변화량 + 집값 변화율 비교
merged_sorted = merged_df.sort_values('date').copy()
merged_sorted['price_change'] = merged_sorted['avg_price'].pct_change() * 100
merged_sorted['sentiment_change'] = merged_sorted['avg_sentiment'].diff()

fig_change = make_subplots(specs=[[{"secondary_y": True}]])

# 감성 변화량 (막대)
colors = ['#4CAF50' if x > 0 else '#F44336' for x in merged_sorted['sentiment_change'].fillna(0)]
fig_change.add_trace(
    go.Bar(
        x=merged_sorted['date'],
        y=merged_sorted['sentiment_change'],
        name='감성 변화량',
        marker_color=colors,
        opacity=0.7
    ),
    secondary_y=False
)

# 집값 변화율 (선)
fig_change.add_trace(
    go.Scatter(
        x=merged_sorted['date'],
        y=merged_sorted['price_change'],
        name='집값 변화율(%)',
        line=dict(color='#1976D2', width=2),
        mode='lines+markers'
    ),
    secondary_y=True
)

fig_change.update_layout(
    title='월별 감성 변화량 vs 집값 변화율',
    template='plotly_white',
    hovermode='x unified',
    height=500
)

fig_change.update_yaxes(title_text="감성 변화량", secondary_y=False)
fig_change.update_yaxes(title_text="집값 변화율 (%)", secondary_y=True)

fig_change.show()

## 8. 구별 민감도 분석

In [ ]:
# 구별 상관계수 계산
regional_correlation = []

for region in REGIONS:
    region_sentiment = regional_sentiment[regional_sentiment['region'] == region][['date', 'avg_sentiment']]
    region_price = price_df[price_df['region'] == region][['date', 'avg_price']]
    
    region_merged = pd.merge(region_sentiment, region_price, on='date', how='inner')
    
    if len(region_merged) >= 5:
        corr, p_val = pearsonr(
            region_merged['avg_sentiment'].dropna(),
            region_merged['avg_price'].dropna()
        )
        
        regional_correlation.append({
            'region': region,
            'correlation': corr,
            'p_value': p_val,
            'sensitivity': abs(corr),
            'significant': '✅' if p_val < 0.05 else ''
        })

sensitivity_df = pd.DataFrame(regional_correlation)
sensitivity_df = sensitivity_df.sort_values('sensitivity', ascending=False)

print("📊 강남3구 뉴스 민감도 비교:")
print(sensitivity_df.to_string(index=False))

if len(sensitivity_df) > 0:
    print(f"\n🏆 가장 민감한 지역: {sensitivity_df.iloc[0]['region']}")

### 8-1. 구별 회귀분석

In [ ]:
# ============================================
# 구별 상세 분석 (회귀분석)
# ============================================

print("="*60)
print("📊 구별 감성-집값 회귀분석")
print("="*60)

regression_results = []

for region in REGIONS:
    region_sent = regional_sentiment[regional_sentiment['region'] == region]
    region_price = price_df[price_df['region'] == region]
    
    region_data = pd.merge(
        region_sent[['date', 'avg_sentiment']],
        region_price[['date', 'avg_price']],
        on='date',
        how='inner'
    )
    
    if len(region_data) > 5:
        slope, intercept, r_value, p_value, std_err = linregress(
            region_data['avg_sentiment'],
            region_data['avg_price']
        )
        
        regression_results.append({
            'region': region,
            'r_squared': r_value**2,
            'slope': slope,
            'p_value': p_value
        })
        
        print(f"\n{region}:")
        print(f"  R² (설명력): {r_value**2:.3f} ({r_value**2*100:.1f}%)")
        print(f"  기울기: {slope:.3f}")
        print(f"  P-value: {p_value:.4f}")
        print(f"  해석: 감성지수 1 증가 시 집값 {slope:.2f}억 {'증가' if slope > 0 else '감소'}")

# 결과 데이터프레임
regression_df = pd.DataFrame(regression_results)

In [ ]:
# 시각화 - R² 비교
if len(regression_df) > 0:
    fig_r2 = px.bar(
        regression_df,
        x='region',
        y='r_squared',
        title='구별 감성-집값 설명력 (R²) 비교',
        text=[f'{r:.1%}' for r in regression_df['r_squared']],
        color='r_squared',
        color_continuous_scale='Blues'
    )
    fig_r2.update_traces(textposition='outside')
    fig_r2.update_layout(template='plotly_white', height=400)
    fig_r2.show()

## 9. 시각화

### 9-1. 이중축 차트 (감성지수 + 집값)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=merged_df['date'],
        y=merged_df['avg_sentiment'],
        name="감성지수",
        line=dict(color='#1976D2', width=2),
        hovertemplate='날짜: %{x}<br>감성지수: %{y:.3f}<extra></extra>'
    ),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(
        x=merged_df['date'],
        y=merged_df['avg_price'],
        name="평균 집값 (억원)",
        line=dict(color='#D32F2F', width=2),
        hovertemplate='날짜: %{x}<br>집값: %{y:.2f}억<extra></extra>'
    ),
    secondary_y=True
)

fig.update_layout(
    title={
        'text': f'뉴스 감성지수 vs 강남3구 집값 추이<br><sup>상관계수: {corr_sentiment_price:.3f}</sup>',
        'x': 0.5
    },
    xaxis_title="날짜",
    hovermode='x unified',
    template='plotly_white',
    legend=dict(x=0.01, y=0.99),
    height=500
)

fig.update_yaxes(title_text="감성지수", secondary_y=False)
fig.update_yaxes(title_text="집값 (억원)", secondary_y=True)

fig.show()

### 9-2. 긍정/부정 뉴스 비교

In [ ]:
monthly_sentiment_dist = news_df.groupby(['year_month', 'sentiment_label']).size().unstack(fill_value=0)
monthly_sentiment_dist = monthly_sentiment_dist.div(monthly_sentiment_dist.sum(axis=1), axis=0) * 100
monthly_sentiment_dist = monthly_sentiment_dist.reset_index()
monthly_sentiment_dist['date'] = monthly_sentiment_dist['year_month'].dt.to_timestamp()

fig = go.Figure()

colors = {'긍정': '#4CAF50', '중립': '#9E9E9E', '부정': '#F44336'}

for sentiment in ['긍정', '중립', '부정']:
    if sentiment in monthly_sentiment_dist.columns:
        fig.add_trace(
            go.Bar(
                x=monthly_sentiment_dist['date'],
                y=monthly_sentiment_dist[sentiment],
                name=sentiment,
                marker_color=colors[sentiment],
                hovertemplate=f'{sentiment}: %{{y:.1f}}%<extra></extra>'
            )
        )

fig.update_layout(
    title='월별 뉴스 감성 분포',
    xaxis_title='날짜',
    yaxis_title='비율 (%)',
    barmode='stack',
    template='plotly_white',
    hovermode='x unified',
    height=450
)

fig.show()

### 9-3. 시차 상관분석 시각화

In [ ]:
fig = go.Figure()

colors = ['#4CAF50' if r > 0 else '#F44336' for r in lag_results['correlation']]
fig.add_trace(
    go.Bar(
        x=lag_results['lag'],
        y=lag_results['correlation'],
        marker_color=colors,
        text=[f'{r:.3f}' for r in lag_results['correlation']],
        textposition='outside',
        hovertemplate='시차: %{x}개월<br>상관계수: %{y:.4f}<extra></extra>'
    )
)

fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)

fig.update_layout(
    title={
        'text': '시차별 상관계수 (감성지수 → 집값)<br><sup>양수 시차 = 감성이 집값에 선행</sup>',
        'x': 0.5
    },
    xaxis_title='시차 (개월)',
    yaxis_title='상관계수',
    template='plotly_white',
    xaxis=dict(tickmode='linear', tick0=-6, dtick=1),
    height=450
)

fig.show()

### 9-4. 구별 민감도 비교

In [ ]:
if len(sensitivity_df) > 0:
    fig = px.bar(
        sensitivity_df,
        x='region',
        y='sensitivity',
        color='correlation',
        color_continuous_scale='RdYlGn',
        color_continuous_midpoint=0,
        title='구별 뉴스 민감도 (상관계수 절대값)',
        labels={'sensitivity': '민감도', 'region': '지역구', 'correlation': '상관계수'},
        text=[f'{s:.3f}' for s in sensitivity_df['sensitivity']]
    )
    fig.update_traces(textposition='outside')
    fig.update_layout(
        template='plotly_white',
        xaxis={'categoryorder': 'total descending'},
        height=450
    )
    fig.show()

### 9-5. 구별 감성지수 vs 집값 산점도

In [ ]:
regional_merged = pd.merge(
    regional_sentiment[['date', 'region', 'avg_sentiment']],
    price_df[['date', 'region', 'avg_price']],
    on=['date', 'region'],
    how='inner'
)

fig = px.scatter(
    regional_merged,
    x='avg_sentiment',
    y='avg_price',
    color='region',
    facet_col='region',
    trendline='ols',
    title='구별 감성지수 vs 집값 관계',
    labels={'avg_sentiment': '감성지수', 'avg_price': '집값 (억원)', 'region': '지역구'},
    color_discrete_sequence=['#1976D2', '#388E3C', '#D32F2F']
)

fig.update_layout(template='plotly_white', showlegend=False, height=400)
fig.show()

## 10. 통합 대시보드

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '① 감성지수 vs 집값 추이',
        '② 시차별 상관계수',
        '③ 월별 감성 분포',
        '④ 구별 민감도'
    ),
    specs=[
        [{"secondary_y": True}, {}],
        [{}, {}]
    ],
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

# ① 이중축 차트
fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['avg_sentiment'], name='감성지수', line=dict(color='#1976D2')),
    row=1, col=1, secondary_y=False
)
fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['avg_price'], name='집값', line=dict(color='#D32F2F')),
    row=1, col=1, secondary_y=True
)

# ② 시차 상관계수
lag_colors = ['#4CAF50' if r > 0 else '#F44336' for r in lag_results['correlation']]
fig.add_trace(
    go.Bar(x=lag_results['lag'], y=lag_results['correlation'], marker_color=lag_colors, showlegend=False),
    row=1, col=2
)

# ③ 긍정/부정 추이
for sentiment, color in [('긍정', '#4CAF50'), ('부정', '#F44336')]:
    if sentiment in monthly_sentiment_dist.columns:
        fig.add_trace(
            go.Scatter(
                x=monthly_sentiment_dist['date'],
                y=monthly_sentiment_dist[sentiment],
                name=sentiment,
                line=dict(color=color)
            ),
            row=2, col=1
        )

# ④ 구별 민감도
if len(sensitivity_df) > 0:
    sens_colors = ['#4CAF50' if c > 0 else '#F44336' for c in sensitivity_df['correlation']]
    fig.add_trace(
        go.Bar(x=sensitivity_df['region'], y=sensitivity_df['sensitivity'], marker_color=sens_colors, showlegend=False),
        row=2, col=2
    )

fig.update_layout(
    height=700,
    title_text='📊 뉴스 감성 영향 분석 대시보드',
    template='plotly_white',
    showlegend=True,
    legend=dict(x=1.02, y=1)
)

fig.show()

## 11. 최종 분석 결과 요약

In [ ]:
# 가장 민감한 지역
if len(sensitivity_df) > 0:
    most_sensitive = sensitivity_df.iloc[0]['region']
    most_sensitive_corr = sensitivity_df.iloc[0]['correlation']
else:
    most_sensitive = 'N/A'
    most_sensitive_corr = 0

print("\n" + "="*70)
print("📊 분석4: 뉴스 감성 영향 분석 - 최종 결과")
print("="*70)

print(f"""
┌─────────────────────────────────────────────────────────────────┐
│  분석 개요                                                       │
├─────────────────────────────────────────────────────────────────┤
│  • 분석 기간: 2023년 1월 ~ 2025년 12월 (3년)                     │
│  • 분석 대상: 강남구, 서초구, 송파구 (강남3구)                    │
│  • 뉴스 출처: 중앙일보 부동산 뉴스                                │
│  • 분석 건수: 뉴스 {len(news_df):,}건                                     │
└─────────────────────────────────────────────────────────────────┘
""")

print(f"""
┌─────────────────────────────────────────────────────────────────┐
│  핵심 질문에 대한 답변                                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Q1. 긍정 뉴스가 많으면 집값이 오르는가?                         │
│  → 상관계수: {corr_sentiment_price:.3f}                                            │""")

if corr_sentiment_price > 0.3:
    print("│  → YES - 양의 상관관계 확인 (긍정 뉴스↑ = 집값↑)                │")
elif corr_sentiment_price > 0:
    print("│  → WEAK - 약한 양의 상관관계                                    │")
else:
    print("│  → NO - 음의 상관관계 또는 관계없음                             │")

print(f"""│                                                                 │
│  Q2. 반응 속도는 얼마나 되는가?                                  │
│  → 최적 시차: {int(best_lag['lag'])}개월                                            │
│  → 뉴스 감성이 {int(best_lag['lag'])}개월 후 집값에 가장 크게 반영                  │
│                                                                 │
│  Q3. 어느 구가 뉴스에 가장 민감한가?                             │
│  → {most_sensitive} (상관계수: {most_sensitive_corr:.3f})                            │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
""")

print(f"""
┌─────────────────────────────────────────────────────────────────┐
│  구별 민감도 순위                                                │
├─────────────────────────────────────────────────────────────────┤""")

for i, row in sensitivity_df.iterrows():
    rank = list(sensitivity_df.index).index(i) + 1
    print(f"│  {rank}위: {row['region']} (상관계수: {row['correlation']:.3f}, 민감도: {row['sensitivity']:.3f})     │")

print("└─────────────────────────────────────────────────────────────────┘")

print(f"""
┌─────────────────────────────────────────────────────────────────┐
│  시사점 및 활용방안                                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. 부동산 시장 예측 지표                                        │
│     → 뉴스 감성지수를 선행지표로 활용 가능                       │
│                                                                 │
│  2. 투자 의사결정 참고                                           │
│     → 긍정 뉴스 증가 시 {int(best_lag['lag'])}개월 후 가격 상승 예상                 │
│                                                                 │
│  3. 지역별 차별화 전략                                           │
│     → {most_sensitive}는 뉴스에 민감, 빠른 대응 필요                     │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
""")

print(f"""
┌─────────────────────────────────────────────────────────────────┐
│  분석의 한계점                                                   │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. 상관관계 ≠ 인과관계                                          │
│     → 뉴스가 집값에 영향 vs 집값이 뉴스에 영향 구분 어려움       │
│                                                                 │
│  2. 키워드 기반 감성분석의 한계                                   │
│     → 문맥 고려 못함 (예: "상승세 꺾여" = 부정적 의미)           │
│                                                                 │
│  3. 단일 매체 분석                                               │
│     → 중앙일보만 분석, 다른 매체 포함 시 결과 달라질 수 있음     │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
""")

## 12. 결과 저장

In [ ]:
# 분석 결과 CSV 저장
output_path = '../data/analysis_results/'
import os
os.makedirs(output_path, exist_ok=True)

# 월별 감성지수 저장
monthly_sentiment.to_csv(f'{output_path}monthly_sentiment.csv', index=False, encoding='utf-8-sig')

# 구별 민감도 저장
sensitivity_df.to_csv(f'{output_path}regional_sensitivity.csv', index=False, encoding='utf-8-sig')

# 시차 분석 결과 저장
lag_results.to_csv(f'{output_path}lag_analysis.csv', index=False, encoding='utf-8-sig')

# 뉴스 감성 결과 저장
news_df.to_csv(f'{output_path}news_with_sentiment.csv', index=False, encoding='utf-8-sig')

# 회귀분석 결과 저장
if len(regression_df) > 0:
    regression_df.to_csv(f'{output_path}regression_results.csv', index=False, encoding='utf-8-sig')

print("✅ 분석 결과가 저장되었습니다!")
print(f"   저장 위치: {output_path}")
print(f"\n   저장된 파일:")
print(f"   - monthly_sentiment.csv (월별 감성지수)")
print(f"   - regional_sensitivity.csv (구별 민감도)")
print(f"   - lag_analysis.csv (시차 분석 결과)")
print(f"   - news_with_sentiment.csv (뉴스 감성 결과)")
print(f"   - regression_results.csv (회귀분석 결과)")